In [93]:
import matplotlib
import networkx
import numpy as np
import sklearn
import seaborn
import torch
import torch_geometric
import torchaudio
import torchvision
import tqdm
import pandas as pd
from collections import defaultdict
from sklearn.preprocessing import LabelEncoder

In [4]:
df = pd.read_csv('E:\dataScience\Fiver orders\Order 4 Author graph\openalex_with_final_merged_authors.csv')

In [7]:
df.sample(5)

,id,title,abstract,keywords,year,authors,references,page_start,page_end,lang,...,Status,Found_By,recovered_authors_openalex,merged_openalex,Match_ON_openalex,Script By,openalex_doi,openalex_year,citedBy,final_merged_authors
13782,6426ed4490e50fcafd4434c0,A Novel Outlier-Tolerable and Predictive Appro...,The Quality-of-Service (QoS) aspects of Web se...,"['Quality of Service (QoS)', 'Service-Oriented...",2022,"[{""id"": ""53f45840dabfaee02ad5fedc"", ""name"": ""X...","['573696126e3b12023e5251a2', '5e2970d3df1a9c0c...",13,29,en,...,By ID,DOI,"[{'id': 'A5102833660', 'name': 'Xiaoning Sun'}...","[{'id': '53f45840dabfaee02ad5fedc', 'name': 'X...",Found,Name,https://doi.org/10.1007/978-3-031-23579-5_2,2022.0,1.0,"[{'aminar_id': '53f45840dabfaee02ad5fedc', 'am..."
3814,56d86c2bdabfae2eeecf9ff6,Complex Elastic Optical Network Simulator (CEO...,This paper presents the features of an advance...,"['elastic optical networks', 'dynamic routing'...",2015,"[{""id"": ""562db59e45cedb3398ec84c3"", ""name"": ""M...","['5488e4c445ce147a86e49025', '53e9a98eb7602d97...",1,4,en,...,By ID,DOI,"[{'id': 'A5002311527', 'name': 'Michał Aibin'}...","[{'id': '562db59e45cedb3398ec84c3', 'name': 'M...",Found,Name,https://doi.org/10.1109/icton.2015.7193519,2015.0,22.0,"[{'aminar_id': '562db59e45cedb3398ec84c3', 'am..."
25267,646629f4d68f896efaa7de45,Unsupervised Fraud Transaction Detection on Dy...,Fraud transaction detection is a pressing need...,"['Unsupervised Anomaly Detection', 'Graph Neur...",2023,"[{""id"": """", ""name"": ""Yangyang Hou"", ""org"": ""An...","['5e2d653a3a55acc837436849', '53e99a48b7602d97...",544,555,en,...,By ID,DOI,"[{'id': 'A5050668789', 'name': 'Yangyang Hou'}...","[{'id': 'A5050668789', 'name': 'Yangyang Hou',...",Found,Name,https://doi.org/10.1007/978-3-031-30678-5_41,2023.0,2.0,"[{'aminar_id': '', 'aminar_name': 'Yangyang Ho..."
7620,5ce2d1e2ced107d4c647917d,Psychological Health Course Teaching Mode Base...,Traditional teaching method cannot really prom...,"['High-order thinking', 'BOPPPS model', 'Psych...",2019,"[{""id"": """", ""name"": ""Zhen Yang"", ""org"": ""Anhui...",[],101,101,en,...,By ID,DOI,"[{'id': 'A5101455301', 'name': 'Zhen Yang'}]","[{'id': 'A5101455301', 'name': 'Zhen Yang', 'o...",Found,Name,https://doi.org/10.3991/ijet.v14i04.10111,2019.0,7.0,"[{'aminar_id': '', 'aminar_name': 'Zhen Yang',..."
3829,58d4cfce0cf221de0360faf0,Numerical Method of Estimating the Maximal Lik...,Consideration was given to the numerical estim...,[],2016,"[{""id"": """", ""name"": ""A. T. Bekishev"", ""org"": ""...",[],1180,1194,en,...,By ID,DOI,"[{'id': 'A5088173991', 'name': 'A. T. Bekishev...","[{'id': 'A5088173991', 'name': 'A. T. Bekishev...",Found,Name,https://doi.org/10.1134/s0005117916070055,2016.0,2.0,"[{'aminar_id': '', 'aminar_name': 'A. T. Bekis..."


In [8]:
df.columns

Index(['id', 'title', 'abstract', 'keywords', 'year', 'authors', 'references',
       'page_start', 'page_end', 'lang', 'volume', 'issue', 'issn', 'isbn',
       'doi', 'url', 'n_citation', 'venue', 'doc_type', 'recovered_authors',
       'authors_merged', 'found_on_website', 'Status', 'Found_By',
       'recovered_authors_openalex', 'merged_openalex', 'Match_ON_openalex',
       'Script By', 'openalex_doi', 'openalex_year', 'citedBy',
       'final_merged_authors'],
      dtype='object')

In [18]:
chunk = df.iloc[:100,:]

In [20]:
chunk.columns == df.columns

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True])

In [24]:
chunk.to_csv('E:\dataScience\Fiver orders\Order 4 Author graph\chunk.csv')

In [26]:
df['final_merged_authors'][0]

"[{'aminar_id': '', 'aminar_name': 'Maria Carolina Novitasari', 'org': 'UCL, London, England', 'orgId': '', 'openalex_id': '', 'openalex_name': ''}, {'aminar_id': '53f455acdabfaefedbb4dfd7', 'aminar_name': 'Johannes Quaas', 'org': 'Univ Leipzig, ScaDS AI, Leipzig, Germany', 'orgId': '', 'openalex_id': '', 'openalex_name': ''}, {'aminar_id': '53f45b10dabfaec09f216a8a', 'aminar_name': 'Miguel R. D. Rodrigues', 'org': 'UCL, London, England', 'orgId': '', 'openalex_id': '', 'openalex_name': ''}]"

In [30]:
df['year']

0        2024
1        2024
2        2024
3        2024
4        2024
         ... 
28427    2020
28428    2023
28429    2017
28430    2023
28431    2019
Name: year, Length: 28432, dtype: int64

In [32]:
df.columns

Index(['id', 'title', 'abstract', 'keywords', 'year', 'authors', 'references',
       'page_start', 'page_end', 'lang', 'volume', 'issue', 'issn', 'isbn',
       'doi', 'url', 'n_citation', 'venue', 'doc_type', 'recovered_authors',
       'authors_merged', 'found_on_website', 'Status', 'Found_By',
       'recovered_authors_openalex', 'merged_openalex', 'Match_ON_openalex',
       'Script By', 'openalex_doi', 'openalex_year', 'citedBy',
       'final_merged_authors'],
      dtype='object')

In [33]:
cols_to_drop = [
    'id', 'page_start', 'page_end', 'lang',
    'volume', 'issue', 'issn', 'isbn',
    'doi', 'url', 'openalex_doi', 'openalex_year',
    'found_on_website', 'Status', 'Found_By', 'Script By',
    'authors', 'recovered_authors', 'authors_merged',
    'recovered_authors_openalex', 'merged_openalex', 'Match_ON_openalex'
]

df = df.drop(columns=cols_to_drop)


In [34]:
df.shape

(28432, 10)

## 🧼 Step: Preprocessing

- ✅ **1.1 Parse `final_merged_authors`**  
  Convert stringified author lists to Python objects.  
  Remove rows with empty or malformed authors.

- ✅ **1.2 Standardize Author IDs**  
  Create a global `author2idx` mapping to assign unique IDs to each author.

- ✅ **1.3 Clean Timestamps**  
  Ensure the `year` column is an integer and contains no missing values.


In [35]:
df.isnull().sum()

title                      0
abstract                 147
keywords                   0
year                       0
references                 0
n_citation                 0
venue                     87
doc_type                  26
citedBy                 1215
final_merged_authors       0
dtype: int64

In [36]:
df = df.dropna().reset_index(drop=True)


In [38]:
df['final_merged_authors'].sample(5)

26763    [{'aminar_id': '53f3532fdabfae4b34951829', 'am...
2940     [{'aminar_id': '', 'aminar_name': 'Amal Chaoui...
26746    [{'aminar_id': '6597cb27e577a45cc2ec23b8', 'am...
24191    [{'aminar_id': '53f430bedabfaee43ebf07f2', 'am...
17165    [{'aminar_id': '', 'aminar_name': 'Chi-Mo Tsai...
Name: final_merged_authors, dtype: object

In [39]:
df['final_merged_authors'][0]

"[{'aminar_id': '', 'aminar_name': 'Ana Petra Jukić', 'org': 'University of Zagreb Faculty of Electrical Engineering and Computing Zagreb', 'orgId': '', 'openalex_id': 'A5030891488', 'openalex_name': 'Ana Jukić'}, {'aminar_id': '647160f07237c4c80acd15c2', 'aminar_name': 'Ana Šelek', 'org': 'University of Zagreb Faculty of Electrical Engineering and Computing Zagreb', 'orgId': '', 'openalex_id': 'A5043595911', 'openalex_name': 'Ana Šelek'}, {'aminar_id': '53f3ac85dabfae4b34b0075b', 'aminar_name': 'Marija Seder', 'org': 'University of Zagreb Faculty of Electrical Engineering and Computing Zagreb', 'orgId': '', 'openalex_id': 'A5081774692', 'openalex_name': 'Marija Seder'}, {'aminar_id': '53f438d4dabfaeecd6975fbe', 'aminar_name': 'Ivana Podnar Žarko', 'org': 'University of Zagreb Faculty of Electrical Engineering and Computing Zagreb', 'orgId': '', 'openalex_id': 'A5098656645', 'openalex_name': 'Ivana Podnar Žarko'}]"

In [40]:
import ast
## converting any string to list
df['final_merged_authors'] = df['final_merged_authors'].apply(ast.literal_eval)


In [41]:
df = df[df['final_merged_authors'].apply(lambda x: isinstance(x, list) and len(x) > 0)]

In [42]:
def has_valid_author(authors):
    return any(a.get('aminar_id') or a.get('aminar_name') for a in authors)

df = df[df['final_merged_authors'].apply(has_valid_author)]

In [51]:
author2idx = {}
idx_counter = 0
missing_ids = []
missing_names = []
for authors in df['final_merged_authors']:
    for author in authors:
        identifier = author.get('aminar_id')
        if not identifier or identifier.strip() == '':
            identifier = author.get('aminar_name')
        if not author.get('aminar_id'):
            #print(f"Missing ID for: {author.get('aminar_name')}")
            missing_ids.append(author.get('aminar_name'))
        if not author.get('aminar_name'):
            #print(f"Missing Name for: {author.get('aminar_id')}")
            missing_names.append(author.get('aminar_id'))
        
        # Skip if identifier is still missing
        if not identifier or identifier.strip() == '':
            continue

        # Assign unique index
        if identifier not in author2idx:
            author2idx[identifier] = idx_counter
            idx_counter += 1


In [50]:
len(missing_ids), len(author2idx)

(37507, 113067)

In [52]:
len(author2idx), len(missing_names)

(113067, 0)

In [47]:
author2idx

{'Ana Petra Jukić': 0,
 '647160f07237c4c80acd15c2': 1,
 '53f3ac85dabfae4b34b0075b': 2,
 '53f438d4dabfaeecd6975fbe': 3,
 '645395c8ca4e0609eeddf13d': 4,
 '666283af63efbd4cc6bd10d6': 5,
 '6372301aec88d95668cb7fa7': 6,
 '65d9feff8a47b622193f63d5': 7,
 '56128cb345ce1e5962c9b71b': 8,
 '645395c8ca4e0609eeddf0c1': 9,
 '562ffd9f45cedb33998afca2': 10,
 '66726c78cfb652168907b053': 11,
 'Peixuan Ding': 12,
 '64fa9d928a47b671e918d7d6': 13,
 '56cb18bec35f4f3c6565fc02': 14,
 'En Huang': 15,
 '62e44585d9f204418d6792f8': 16,
 'Yanan Zhang': 17,
 'Bing Chen': 18,
 '542e0e7ddabfae11fc4c0227': 19,
 '56169caf45cedb3397b572a0': 20,
 'Zbigniew Szkulmowski': 21,
 '53f42df9dabfaee02ac68811': 22,
 'Michal Joachimiak': 23,
 '562d33dd45cedb3398d88708': 24,
 'Marcin Schiller': 25,
 '53f46a8bdabfaeecd6a1d200': 26,
 '640401b47691d561fb2da3e5': 27,
 '562ba91645cedb3398af4473': 28,
 '5627577545cedb33986f132b': 29,
 'Michael Tang': 30,
 'Yubing Liu': 31,
 'Noel Wong': 32,
 '653fe7ca50dee4c4228bbb30': 33,
 '63156db8cd72

In [53]:
author2idx = {}
idx_counter = 0
##new author2idx with name as identifier
for authors in df['final_merged_authors']:
    for author in authors:
        name = author.get('aminar_name')
        if name and name not in author2idx:
            author2idx[name] = idx_counter
            idx_counter += 1

In [55]:
for author in df['final_merged_authors'][0]:
    print(author)

{'aminar_id': '', 'aminar_name': 'Ana Petra Jukić', 'org': 'University of Zagreb Faculty of Electrical Engineering and Computing Zagreb', 'orgId': '', 'openalex_id': 'A5030891488', 'openalex_name': 'Ana Jukić'}
{'aminar_id': '647160f07237c4c80acd15c2', 'aminar_name': 'Ana Šelek', 'org': 'University of Zagreb Faculty of Electrical Engineering and Computing Zagreb', 'orgId': '', 'openalex_id': 'A5043595911', 'openalex_name': 'Ana Šelek'}
{'aminar_id': '53f3ac85dabfae4b34b0075b', 'aminar_name': 'Marija Seder', 'org': 'University of Zagreb Faculty of Electrical Engineering and Computing Zagreb', 'orgId': '', 'openalex_id': 'A5081774692', 'openalex_name': 'Marija Seder'}
{'aminar_id': '53f438d4dabfaeecd6975fbe', 'aminar_name': 'Ivana Podnar Žarko', 'org': 'University of Zagreb Faculty of Electrical Engineering and Computing Zagreb', 'orgId': '', 'openalex_id': 'A5098656645', 'openalex_name': 'Ivana Podnar Žarko'}


In [60]:
author2idx

{'Ana Petra Jukić': 0,
 'Ana Šelek': 1,
 'Marija Seder': 2,
 'Ivana Podnar Žarko': 3,
 'Isabel Sophie Fitton': 4,
 'Elizabeth Dark': 5,
 'Manoela Milena Oliveira da Silva': 6,
 'Jeremy Dalton': 7,
 'Michael J Proulx': 8,
 'Christopher Clarke': 9,
 'Christof Lutteroth': 10,
 'Xinyi Yu': 11,
 'Peixuan Ding': 12,
 'Linlin Ou': 13,
 'Weidong Zhang': 14,
 'En Huang': 15,
 'Maomao Zhang': 16,
 'Yanan Zhang': 17,
 'Bing Chen': 18,
 'Lihui Peng': 19,
 'Yi Li': 20,
 'Zbigniew Szkulmowski': 21,
 'Slawomir Grzelak': 22,
 'Michal Joachimiak': 23,
 'Sebastian Meszynski': 24,
 'Marcin Schiller': 25,
 'Oleksandr Sokolov': 26,
 'Duran Bao': 27,
 'Qingbo Shu': 28,
 'Bo Ning': 29,
 'Michael Tang': 30,
 'Yubing Liu': 31,
 'Noel Wong': 32,
 'Zhengming Ding': 33,
 'Zizhan Zheng': 34,
 'Christopher J. Lyon': 35,
 'Tony Hu': 36,
 'Jia Fan': 37,
 'Enlai Pu': 38,
 'Fei Wang': 39,
 'Zongliang Yang': 40,
 'Jun Wang': 41,
 'Zhiming Li': 42,
 'Xin Huang': 43,
 'Kun Zhao': 44,
 'Shiwen Zeng': 45,
 'Lijian Zhou': 46

In [61]:

df = df[df['year'].notnull()]
df['year'] = df['year'].astype(int)



In [62]:
df.shape

(27041, 10)

✅ **Step 1 Complete:** Data preprocessing finished — authors parsed, IDs standardized, and timestamps cleaned. Ready to build the temporal graph!


<style>
  body { background-color: #121212; color: #e0e0e0; font-family: sans-serif; }
  code { background-color: #1e1e1e; color: #dcdcaa; padding: 2px 4px; border-radius: 4px; }
</style>

## 🔗 Step 2: Temporal Edge Construction

- ✅ Loop through each row (paper) in the dataset.
- ✅ Extract co-author pairs from `final_merged_authors`.
- ✅ Use `author2idx` to convert names to global node indices.
- ✅ Create undirected edges for each co-author pair.
- ✅ Attach `year` as the timestamp for each edge.
- ✅ Optionally store `citedBy`, `doc_type`, or `venue` as edge attributes.


In [ ]:
print(df['doc_type'].unique())
le_doc = LabelEncoder()
df['doc_type'] = le_doc.fit_transform(df['doc_type'])

['Journal' 'Conference']


In [73]:
#Initialize empty lists
#Loop through each row of df
#Extract author names → convert to indices using author2idx
#Create all author pairs (combinations)
#Add both directions (A→B and B→A)
#Collect year and other edge features
#Convert everything to torch.tensor
edge_index = []
edge_timestamps= []
edge_attributes = []
for _,row in df.iterrows():
    authors = row['final_merged_authors']
    year = row['year']
    cited_by = row['citedBy']  # 🔧 FIXED
    doc_type = row['doc_type']
    author_indices = [author2idx[author['aminar_name']] for author in authors if author['aminar_name'] in author2idx]
    for i in range(len(author_indices)):
        for j in range(i + 1, len(author_indices)):
            a,b = author_indices[i], author_indices[j]
            edge_index.append([a, b])
            edge_index.append([b, a])  # Add reverse direction
            edge_timestamps.append(year)
            edge_timestamps.append(year)  # Add reverse direction timestamp
            edge_attributes.append([cited_by, doc_type])
            edge_attributes.append([cited_by, doc_type])
    

In [75]:
print(edge_index[0:10])
print(edge_timestamps[0:10])
print(edge_attributes[0:10])

[[0, 1], [1, 0], [0, 2], [2, 0], [0, 3], [3, 0], [1, 2], [2, 1], [1, 3], [3, 1]]
[2024, 2024, 2024, 2024, 2024, 2024, 2024, 2024, 2024, 2024]
[[0.0, 1], [0.0, 1], [0.0, 1], [0.0, 1], [0.0, 1], [0.0, 1], [0.0, 1], [0.0, 1], [0.0, 1], [0.0, 1]]


In [76]:
edge_index = torch.tensor(edge_index, dtype=torch.long).t().contiguous()
edge_timestamps = torch.tensor(edge_timestamps, dtype=torch.float)  
edge_attributes = torch.tensor(edge_attributes, dtype=torch.float)

In [65]:
author_indices

[0, 1, 2, 3]

In [78]:
 len(author2idx)

106516

In [90]:
df.columns

Index(['title', 'abstract', 'keywords', 'year', 'references', 'n_citation',
       'venue', 'doc_type', 'citedBy', 'final_merged_authors'],
      dtype='object')

In [95]:
df['citedBy']

0         0.0
1         4.0
2         2.0
3         1.0
4         0.0
         ... 
27036     2.0
27037     0.0
27038     8.0
27039    17.0
27040     2.0
Name: citedBy, Length: 27041, dtype: float64

In [97]:
len(author2idx)

106516

In [96]:
df['final_merged_authors'][0]

[{'aminar_id': '',
  'aminar_name': 'Ana Petra Jukić',
  'org': 'University of Zagreb Faculty of Electrical Engineering and Computing Zagreb',
  'orgId': '',
  'openalex_id': 'A5030891488',
  'openalex_name': 'Ana Jukić'},
 {'aminar_id': '647160f07237c4c80acd15c2',
  'aminar_name': 'Ana Šelek',
  'org': 'University of Zagreb Faculty of Electrical Engineering and Computing Zagreb',
  'orgId': '',
  'openalex_id': 'A5043595911',
  'openalex_name': 'Ana Šelek'},
 {'aminar_id': '53f3ac85dabfae4b34b0075b',
  'aminar_name': 'Marija Seder',
  'org': 'University of Zagreb Faculty of Electrical Engineering and Computing Zagreb',
  'orgId': '',
  'openalex_id': 'A5081774692',
  'openalex_name': 'Marija Seder'},
 {'aminar_id': '53f438d4dabfaeecd6975fbe',
  'aminar_name': 'Ivana Podnar Žarko',
  'org': 'University of Zagreb Faculty of Electrical Engineering and Computing Zagreb',
  'orgId': '',
  'openalex_id': 'A5098656645',
  'openalex_name': 'Ivana Podnar Žarko'}]

In [ ]:
#Total number of papers authored
#Total citations received (sum of citedBy)
#Active years (min + max year)
#Optional: degree, average citation, organization encoding
num_nodes = len(author2idx)
num_papers = defaultdict(int)
num_citations = defaultdict(int)
min_years = defaultdict(lambda: float('inf'))
max_years = defaultdict(lambda: float('-inf'))
for _,row in df.iterrows():
    authors = row['final_merged_authors']
    cited_by = row['citedBy']
    year = row['year']
    for author in authors:
        name = author['aminar_name']
        if name in author2idx:
            continue
        idx = author2idx[name]
        num_papers[idx] += 1
        num_citations[idx] += cited_by
        min_years[idx] = min(min_years[idx], year)
        max_years[idx] = max(max_years[idx], year)

In [ ]:
features = []
for idx in range(num_nodes):
    papers = num_papers[idx]
    citation = num_citations[idx]
    active_years = max_years[idx] - min_years[idx] + 1
    min_year = min_years[idx] if min_years[idx] != float('inf') else 0
    max_year = max_years[idx] if max_years[idx] != float('-inf') else 0
    avg_citations = num_citations / num_papers if num_papers > 0 else 0
    features.append([papers, citation, active_years, min_year, max_year, avg_citations])
    

In [92]:
print(num_papers)

[1, 1, 1, 1, 1]


In [83]:
df['keywords'].nunique()

24983